In [ ]:
!wget -O RDS-2020-0016_California.zip "https://usfs-public.box.com/shared/static/v1jx1w94w3dswhcs1jvsyj9sukccftdm.zip?download=1"


--2025-03-27 17:31:52--  https://usfs-public.box.com/shared/static/v1jx1w94w3dswhcs1jvsyj9sukccftdm.zip?download=1
Resolving usfs-public.box.com (usfs-public.box.com)... 74.112.186.157, 2620:117:bff0:12d::
Connecting to usfs-public.box.com (usfs-public.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/v1jx1w94w3dswhcs1jvsyj9sukccftdm.zip [following]
--2025-03-27 17:31:52--  https://usfs-public.box.com/public/static/v1jx1w94w3dswhcs1jvsyj9sukccftdm.zip
Reusing existing connection to usfs-public.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://usfs-public.app.box.com/public/static/v1jx1w94w3dswhcs1jvsyj9sukccftdm.zip [following]
--2025-03-27 17:31:52--  https://usfs-public.app.box.com/public/static/v1jx1w94w3dswhcs1jvsyj9sukccftdm.zip
Resolving usfs-public.app.box.com (usfs-public.app.box.com)... 74.112.186.157, 2620:117:bff0:12d::
Connecting to usfs-public.app.box.

In [ ]:
pip install pyproj


## Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
from sklearn.neighbors import KDTree
import pandas as pd
import os
import random
import rasterio
import xml.etree.ElementTree as ET

Mounted at /content/drive


In [ ]:
calfire_df = pd.read_csv('calfire_cimis_completed_rows.csv')
calfire_coords = calfire_df[['incident_latitude', 'incident_longitude']]

In [ ]:
calfire_coords

,incident_latitude,incident_longitude
0,40.774000,-122.309000
1,33.510716,-115.916687
2,37.037760,-121.006939
3,33.762600,-117.727400
4,41.443500,-120.870540
...,...,...
1579,35.817780,-120.097150
1580,41.853000,-123.676000
1581,33.119448,-116.677511
1582,33.450600,-116.857817


In [ ]:

def find_closest_points(calfire_coords, big_csv_path, lat_col, lon_col, chunksize=100000):
    closest_matches = []
    calfire_coords_rad = np.radians(calfire_coords)

    for chunk in pd.read_csv(big_csv_path, usecols=[lat_col, lon_col], chunksize=chunksize):
        chunk_clean = chunk.dropna()
        chunk_rad = np.radians(chunk_clean[[lat_col, lon_col]])
        tree = KDTree(chunk_rad, metric='haversine')

        dist, idx = tree.query(calfire_coords_rad, k=1)
        # Convert to km (Earth radius = 6371 km)
        closest_chunk_matches = chunk_clean.iloc[idx.flatten()].copy()
        closest_chunk_matches['distance_km'] = dist.flatten() * 6371
        closest_matches.append(closest_chunk_matches.reset_index(drop=True))

    # Keep only the closest across all chunks
    all_closest = pd.concat(closest_matches, axis=0)
    all_closest_grouped = all_closest.groupby(all_closest.index).apply(lambda g: g.loc[g['distance_km'].idxmin()])
    return all_closest_grouped.reset_index(drop=True)



big_csv_path = '/content/drive/MyDrive/path_to_your_big_file.csv'
lat_col = 'latitude'  # Replace with actual column name
lon_col = 'longitude'

#closest_refs = find_closest_points(calfire_coords, big_csv_path, lat_col, lon_col)


big_csv_path = '/content/drive/MyDrive/UploadedFiles/wildfire_risk_dataset.csv'
sample_df = pd.read_csv(big_csv_path, nrows=100)  # change nrows as needed
print(sample_df.head())




           x          y        date  burn_probability  \
0 -2286960.0  2452710.0  2015-01-01          0.000000   
1 -2286960.0  2452680.0  2015-01-01          0.000000   
2 -2286930.0  2452680.0  2015-01-01          0.000516   
3 -2286900.0  2452680.0  2015-01-01          0.001066   
4 -2286960.0  2452650.0  2015-01-01          0.000000   

   conditional_flame_length  conditional_risk_to_structures  exposure  \
0                  0.000000                         0.00000  0.000000   
1                  0.000000                         0.00000  0.000000   
2                  3.608344                        47.49138  0.465494   
3                  4.127649                        51.63526  1.000000   
4                  0.000000                         0.00000  0.000000   

   flame_length_exceedance_4ft  flame_length_exceedance_8ft  \
0                     0.000000                     0.000000   
1                     0.000000                     0.000000   
2                     0.48825

In [ ]:
from pyproj import Transformer
import pandas as pd

# Example sample
sample_df = pd.read_csv(big_csv_path, nrows=5)

# Most likely projection: NAD83 / California Albers (EPSG:3310)
transformer = Transformer.from_crs("EPSG:3310", "EPSG:4326", always_xy=True)

# Convert all x, y
sample_df['longitude'], sample_df['latitude'] = transformer.transform(
    sample_df['x'].values, sample_df['y'].values
)

print(sample_df[['x', 'y', 'longitude', 'latitude']].head())


           x          y   longitude   latitude
0 -2286960.0  2452710.0 -155.277094  56.631515
1 -2286960.0  2452680.0 -155.276930  56.631244
2 -2286930.0  2452680.0 -155.276509  56.631349
3 -2286900.0  2452680.0 -155.276088  56.631455
4 -2286960.0  2452650.0 -155.276766  56.630972


In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from pyproj import Transformer

# Load CAL FIRE data
calfire_df = pd.read_csv('calfire_cimis_completed_rows.csv')
calfire_coords = calfire_df[['incident_latitude', 'incident_longitude']]
calfire_coords_rad = np.radians(calfire_coords)

# Set up transformer: EPSG:3310 → WGS84 (lon, lat)
transformer = Transformer.from_crs("EPSG:3310", "EPSG:4326", always_xy=True)

def find_closest_points_with_xy(calfire_coords_rad, big_csv_path, x_col, y_col, chunksize=100000):
    closest_matches = []

    # Columns to keep from the wildfire risk dataset
    cols_to_use = [
        x_col, y_col,
        'date',
        'burn_probability',
        'conditional_flame_length',
        'conditional_risk_to_structures',
        'exposure',
        'flame_length_exceedance_4ft',
        'flame_length_exceedance_8ft',
        'risk_to_structures',
        'wildfire_hazard_potential'
    ]

    for chunk in pd.read_csv(big_csv_path, usecols=cols_to_use, chunksize=chunksize):
        chunk = chunk.dropna(subset=[x_col, y_col])

        # Convert x, y to lon/lat
        chunk['longitude'], chunk['latitude'] = transformer.transform(
            chunk[x_col].values, chunk[y_col].values
        )
        chunk_coords = chunk[['latitude', 'longitude']].to_numpy()
        chunk_coords_rad = np.radians(chunk_coords)

        # Build BallTree and query closest CAL FIRE incident
        tree = BallTree(chunk_coords_rad, metric='haversine')
        dist, idx = tree.query(calfire_coords_rad, k=1)

        # Get matched rows
        matched_rows = chunk.iloc[idx.flatten()].copy()
        matched_rows['distance_km'] = dist.flatten() * 6371
        closest_matches.append(matched_rows.reset_index(drop=True))

    # Keep only the closest match per CAL FIRE point
    all_matches = pd.concat(closest_matches, axis=0)
    closest_final = all_matches.groupby(all_matches.index).apply(
        lambda g: g.loc[g['distance_km'].idxmin()]
    )
    return closest_final.reset_index(drop=True)

# Run the matching function
big_csv_path = '/content/drive/MyDrive/UploadedFiles/wildfire_risk_dataset.csv'
x_col = 'x'
y_col = 'y'

closest_refs = find_closest_points_with_xy(calfire_coords_rad, big_csv_path, x_col, y_col)

# Combine CAL FIRE data with matched wildfire risk data
final_df = pd.concat([calfire_df.reset_index(drop=True), closest_refs], axis=1)


# After our feature addition

In [47]:
final_df.columns

Index(['incident_id', 'incident_name', 'incident_date_last_update',
       'incident_date_created', 'incident_acres_burned', 'incident_longitude',
       'incident_latitude', 'incident_dateonly_created', 'DayAirTmpAvg01',
       'DayPrecip01', 'DayRelHumAvg01', 'DaySoilTmpAvg01', 'DayWindSpdAvg01',
       'DayAirTmpAvg02', 'DayPrecip02', 'DayRelHumAvg02', 'DaySoilTmpAvg02',
       'DayWindSpdAvg02', 'DayAirTmpAvg03', 'DayPrecip03', 'DayRelHumAvg03',
       'DaySoilTmpAvg03', 'DayWindSpdAvg03', 'DayAirTmpAvg04', 'DayPrecip04',
       'DayRelHumAvg04', 'DaySoilTmpAvg04', 'DayWindSpdAvg04',
       'DayAirTmpAvg05', 'DayPrecip05', 'DayRelHumAvg05', 'DaySoilTmpAvg05',
       'DayWindSpdAvg05', 'DayAirTmpAvg06', 'DayPrecip06', 'DayRelHumAvg06',
       'DaySoilTmpAvg06', 'DayWindSpdAvg06', 'DayAirTmpAvg07', 'DayPrecip07',
       'DayRelHumAvg07', 'DaySoilTmpAvg07', 'DayWindSpdAvg07',
       'DayAirTmpAvg08', 'DayPrecip08', 'DayRelHumAvg08', 'DaySoilTmpAvg08',
       'DayWindSpdAvg08', 'DayAir

In [48]:
final_df.dropna(axis=0)['flame_length_exceedance_4ft']

,flame_length_exceedance_4ft
0,0.720917
1,0.782333
2,0.883381
3,1.000000
4,0.980533
...,...
1579,0.785240
1580,0.784654
1581,0.675252
1582,0.732321


## List of files

In [ ]:
# Files of interest
raster_files = [
    "BP_CA.tif",
    "CFL_CA.tif",
    "CRPS_CA.tif",
    "Exposure_CA.tif",
    "FLEP4_CA.tif",
    "FLEP8_CA.tif",
    "RPS_CA.tif",
    "WHP_CA.tif"
]
folder = "RDS_data/CA"
for filename in raster_files:
    path = os.path.join(folder, filename)
    with rasterio.open(path) as src:
        data = src.read(1)  # Read the first (and usually only) band
        masked_data = np.ma.masked_equal(data, src.nodata if src.nodata is not None else -9999)

        # Clean stats
        print(f"📂 {filename}")
        print(f"  Shape         : {data.shape}")
        print(f"  CRS           : {src.crs}")
        print(f"  Resolution    : {src.res}")
        print(f"  Min           : {masked_data.min():.4f}")
        print(f"  Max           : {masked_data.max():.4f}")
        print(f"  Mean          : {masked_data.mean():.4f}")

        # Print 5 random samples
        print("  🔹 Sample values:")
        h, w = data.shape
        for _ in range(5):
            row, col = random.randint(0, h-1), random.randint(0, w-1)
            value = data[row, col]
            print(f"    At ({row}, {col}) -> {value}")
        print("-" * 50)

📂 BP_CA.tif
  Shape         : (40346, 23649)
  CRS           : EPSG:5070
  Resolution    : (30.0, 30.0)
  Min           : 0.0000
  Max           : 0.0912
  Mean          : 0.0057
  🔹 Sample values:
    At (19034, 2048) -> -3.3999999521443642e+38
    At (599, 8877) -> -3.3999999521443642e+38
    At (2366, 18262) -> -3.3999999521443642e+38
    At (34691, 21547) -> 0.0024628478568047285
    At (389, 3474) -> 0.011612153612077236
--------------------------------------------------
📂 CFL_CA.tif
  Shape         : (40346, 23649)
  CRS           : EPSG:5070
  Resolution    : (30.0, 30.0)
  Min           : 0.0000
  Max           : 48.2316
  Mean          : 5.4923
  🔹 Sample values:
    At (2818, 13791) -> -3.3999999521443642e+38
    At (10967, 706) -> 9.798707962036133
    At (23681, 15833) -> 5.767733573913574
    At (30228, 12023) -> 0.0
    At (1431, 15521) -> -3.3999999521443642e+38
--------------------------------------------------
📂 CRPS_CA.tif
  Shape         : (40346, 23649)
  CRS       

## Data summaries

In [ ]:


folder = "RDS_data/CA"

raster_files = [
    "BP_CA.tif",
    "CFL_CA.tif",
    "CRPS_CA.tif",
    "Exposure_CA.tif",
    "FLEP4_CA.tif",
    "FLEP8_CA.tif",
    "RPS_CA.tif",
    "WHP_CA.tif"
]

for filename in raster_files:
    path = os.path.join(folder, filename)
    with rasterio.open(path) as src:
        data = src.read(1)


        nodata_val = src.nodata
        if nodata_val is None:
            # Use a default float32 nodata sentinel (common in fire datasets)
            nodata_val = -3.4e+38

        # Mask invalid data
        masked = np.ma.masked_values(data, nodata_val)

        print(f"📂 {filename}")
        print(f"  Shape         : {data.shape}")
        print(f"  CRS           : {src.crs}")
        print(f"  Resolution    : {src.res}")
        print(f"  NoData Value  : {nodata_val}")
        print(f"  Min           : {masked.min():.4f}")
        print(f"  Max           : {masked.max():.4f}")
        print(f"  Mean          : {masked.mean():.4f}")

        # Sample valid (unmasked) pixels
        valid_coords = np.argwhere(~masked.mask)
        print("  🔹 Sample values:")
        for _ in range(5):
            row, col = random.choice(valid_coords)
            print(f"    At ({row}, {col}) -> {data[row, col]}")
        print("-" * 50)


📂 BP_CA.tif
  Shape         : (40346, 23649)
  CRS           : EPSG:5070
  Resolution    : (30.0, 30.0)
  NoData Value  : -3.3999999521443642e+38
  Min           : 0.0000
  Max           : 0.0912
  Mean          : 0.0057
  🔹 Sample values:
    At (2593, 5902) -> 0.010935667902231216
    At (31362, 15460) -> 0.0010397735750302672
    At (12118, 10957) -> 0.0077599454671144485
    At (14453, 2109) -> 0.005071374122053385
    At (3761, 1472) -> 0.0005698921740986407
--------------------------------------------------
📂 CFL_CA.tif
  Shape         : (40346, 23649)
  CRS           : EPSG:5070
  Resolution    : (30.0, 30.0)
  NoData Value  : -3.3999999521443642e+38
  Min           : 0.0000
  Max           : 48.2316
  Mean          : 5.4923
  🔹 Sample values:
    At (3896, 8725) -> 9.712959289550781
    At (3432, 6903) -> 6.602468967437744
    At (27194, 12185) -> 0.0
    At (28850, 14532) -> 5.769778251647949
    At (32098, 18988) -> 7.770827770233154
------------------------------------------

## File descriptions

In [ ]:
# Folder containing the .xml files
folder = "RDS_data/CA"

# Keywords to look for
keywords = ["date", "time", "year", "temporal", "summary", "abstract", "source", "title", "description"]

# List all XML files
xml_files = [f for f in os.listdir(folder) if f.endswith(".tif.xml")]

def search_metadata(xml_path):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        print(f"\n📄 Metadata from: {os.path.basename(xml_path)}")
        print("-" * 50)

        found_any = False
        for elem in root.iter():
            for keyword in keywords:
                if keyword.lower() in elem.tag.lower() or \
                   (elem.text and keyword.lower() in elem.text.lower()):
                    found_any = True
                    print(f"🔹 Tag: {elem.tag}")
                    print(f"    Text: {elem.text.strip() if elem.text else '[None]'}")
        if not found_any:
            print("⚠️ No matching metadata found for keywords.")
    except Exception as e:
        print(f"❌ Failed to parse {xml_path}: {e}")

# Run the metadata check for each .xml
for xml_file in xml_files:
    full_path = os.path.join(folder, xml_file)
    search_metadata(full_path)



📄 Metadata from: WHP_CA.tif.xml
--------------------------------------------------
🔹 Tag: pubdate
    Text: 2020
🔹 Tag: title
    Text: Wildfire Risk to Communities: Spatial datasets of landscape-wide wildfire risk components for the United States
🔹 Tag: purpose
    Text: The geospatial data products described and distributed here are part of the Wildfire Risk to Communities project. This project was directed by Congress in the 2018 Consolidated Appropriations Act (i.e., 2018 Omnibus Act, H.R. 1625, Section 210: Wildfire Hazard Severity Mapping) to help U.S. communities understand components of their relative wildfire risk profile, the nature and effects of wildfire risk, and actions communities can take to mitigate risk. These data represent the first time wildfire risk to communities has been mapped nationally with consistent methodology. They provide foundational information for comparing the relative wildfire risk among populated communities in the United States.
🔹 Tag: purpose
  

In [ ]:
import rasterio
import os
import csv
from tqdm import tqdm

# Setup
data_dir = "RDS_data/CA"
output_csv = "wildfire_risk_dataset.csv"

files = {
    "BP_CA.tif": "burn_probability",
    "CFL_CA.tif": "conditional_flame_length",
    "CRPS_CA.tif": "conditional_risk_to_structures",
    "Exposure_CA.tif": "exposure",
    "FLEP4_CA.tif": "flame_length_exceedance_4ft",
    "FLEP8_CA.tif": "flame_length_exceedance_8ft",
    "RPS_CA.tif": "risk_to_structures",
    "WHP_CA.tif": "wildfire_hazard_potential"
}

# Open all rasters
rasters = {}
transforms = {}
nodata_vals = {}

for fname, var in files.items():
    src = rasterio.open(os.path.join(data_dir, fname))
    rasters[var] = src.read(1)  # read once
    transforms[var] = src.transform
    nodata_vals[var] = src.nodata

rows, cols = rasters["burn_probability"].shape  # assume same shape

# Write to CSV row-by-row
with open(output_csv, mode="w", newline="") as f:
    writer = csv.writer(f)
    header = ["x", "y", "date"] + list(files.values())
    writer.writerow(header)

    for row in tqdm(range(rows)):
        for col in range(cols):
            record = {}
            skip = False
            for var, data in rasters.items():
                val = data[row, col]
                if val == nodata_vals[var] or val < -1e10:
                    skip = True
                    break
                record[var] = val

            if skip:
                continue

            # Get x, y coords from one transform
            transform = list(transforms.values())[0]
            x, y = transform * (col + 0.5, row + 0.5)

            writer.writerow([
                round(x, 5),
                round(y, 5),
                "2015-01-01"
            ] + [record[var] for var in files.values()])

print(f"Saved to {output_csv}")


100%|██████████| 40346/40346 [1:52:09<00:00,  6.00it/s]


Saved to wildfire_risk_dataset.csv


## List of all files

In [ ]:
!ls RDS_data/CA

BP_CA.tif	CFL_CA.tif.xml	 Exposure_CA.tif.ovr  FLEP8_CA.tif	RPS_CA.tif.xml
BP_CA.tif.ovr	CRPS_CA.tif	 Exposure_CA.tif.xml  FLEP8_CA.tif.ovr	WHP_CA.tif
BP_CA.tif.xml	CRPS_CA.tif.ovr  FLEP4_CA.tif	      FLEP8_CA.tif.xml	WHP_CA.tif.ovr
CFL_CA.tif	CRPS_CA.tif.xml  FLEP4_CA.tif.ovr     RPS_CA.tif	WHP_CA.tif.xml
CFL_CA.tif.ovr	Exposure_CA.tif  FLEP4_CA.tif.xml     RPS_CA.tif.ovr
